In [9]:
!pip install --upgrade -q langchain-pinecone langchain-google-genai langchain_community pypdf PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.2 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
from pinecone import Pinecone ,  ServerlessSpec
pinecone_key = userdata.get("PINECONE_KEY")
pc = Pinecone(api_key=pinecone_key)

In [6]:
index_name = "my-rag-project"
pc.delete_index(index_name)

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
index = pc.Index(index_name)

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
embeddings= GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the PDF using PyPDFLoader
loader = PyPDFLoader("/content/SDNEA2024001.pdf")
documents = loader.load_and_split()  # Returns a list of Document objects

In [11]:
documents

[Document(metadata={'source': '/content/SDNEA2024001.pdf', 'page': 0}, page_content='IMF Staff Discussion Notes (SDNs) showcase \npolicy-related analysis and research being \ndeveloped by IMF staff members and are \npublished to elicit comments and to encourage \ndebate. The views expressed in Staff Discussion \nNotes are those of the author(s) and do not \nnecessarily represent the views of the IMF, \nits Executive Board, or IMF management. \n \n2024 \nJAN \n \n \nGen-AI: Artificial \nIntelligence and the \nFuture of Work  \nPrepared by Mauro Cazzaniga, Florence Jaumotte, Longji Li, \nGiovanni Melina, Augustus J. Panton, Carlo Pizzinelli, Emma \nRockall, and Marina M. Tavares  \nSDN/2024/001'),
 Document(metadata={'source': '/content/SDNEA2024001.pdf', 'page': 1}, page_content='-  \n \n©2024 International Monetary Fund SDN/2024/001 \nIMF Staff Discussion Notes \nResearch Department \n \nGen-AI: Artificial Intelligence and the Future of Work  \n \nPrepared by Mauro Cazzaniga, Florence 

In [12]:
pages = [doc.page_content for doc in documents]
pages

['IMF Staff Discussion Notes (SDNs) showcase \npolicy-related analysis and research being \ndeveloped by IMF staff members and are \npublished to elicit comments and to encourage \ndebate. The views expressed in Staff Discussion \nNotes are those of the author(s) and do not \nnecessarily represent the views of the IMF, \nits Executive Board, or IMF management. \n \n2024 \nJAN \n \n \nGen-AI: Artificial \nIntelligence and the \nFuture of Work  \nPrepared by Mauro Cazzaniga, Florence Jaumotte, Longji Li, \nGiovanni Melina, Augustus J. Panton, Carlo Pizzinelli, Emma \nRockall, and Marina M. Tavares  \nSDN/2024/001',
 '-  \n \n©2024 International Monetary Fund SDN/2024/001 \nIMF Staff Discussion Notes \nResearch Department \n \nGen-AI: Artificial Intelligence and the Future of Work  \n \nPrepared by Mauro Cazzaniga, Florence Jaumotte, Longji Li, Giovanni Melina, Augustus J. Panton, \nCarlo Pizzinelli, Emma Rockall, and Marina M. Tavares* \n \nAuthorized for distribution by Pierre-Olivier G

In [1]:
# import google.generativeai as genai

# genai.configure(api_key="api_key")
# models = genai.list_models()

# for model in models:
#     print(model.name)


#  i use it for just check google models

In [13]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings
)

In [14]:
vector_store.add_documents(documents=documents)


['587684d7-8398-46dc-b20b-8241f1d2225c',
 '48cdb50d-a581-467f-a740-27d299059ed5',
 '1fa38ac2-4d2c-4696-b194-abd96312331e',
 '8d2b25b0-74d7-4386-8276-3a24c14d5748',
 'ad228614-34ef-4b3c-8a5b-7ef5fd351b2f',
 'a00cc0d8-a6d8-4967-a38f-36bf9edb79be',
 '2453c2f0-41f2-464d-b73e-cc5ce4c4192e',
 'b12b9e7e-b78d-45d7-b08f-f143d174036b',
 'b97217a5-475c-407c-b176-99910581e8cb',
 '70deb323-0524-46d5-af13-d4d4413baca2',
 'a6ddd479-541d-47c1-9102-af57c916c633',
 'f94aeb16-114f-47e2-95a0-3953953bd166',
 '3edf0cdb-df43-462f-8729-7346df5d9afe',
 'bc76bd89-0fb1-4502-a624-4c65e2fe48d2',
 '5b2c9ff8-412c-4289-be18-6fcda6637130',
 'd3725d6b-e522-4b8e-8e74-396f36313c15',
 'd250f005-cba8-45b0-bbe6-8b951a1411d3',
 'd4739ccd-2e26-4955-8d12-cb1299660cb4',
 'a2b601dd-e3fb-4c04-b439-cee6486d62bc',
 '96457ff9-816b-41f5-bc58-ad9966d80334',
 '02ebb3cc-fbbe-4ac5-abf5-51355e74622e',
 '7570ca48-1436-4e79-a0fc-ba23c6d2bbe5',
 '2e73c51f-dfba-40d9-b6e5-667542df8a0a',
 '6c8e2b30-b639-4658-9f69-29c9898f6e16',
 '6017ca1c-5e4d-

In [15]:
# Initialize the CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Split the text into chunks
chunks = text_splitter.create_documents(pages)

# Display results
print(f"Number of chunks: {len(chunks)}")
print(f"Type of first chunk: {type(chunks[0])}")

Number of chunks: 51
Type of first chunk: <class 'langchain_core.documents.base.Document'>


In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

In [17]:
def user_answer(query:str):
    vector_result = vector_store.similarity_search(query, k=2)

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        temprature=0.7,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

    prompt = PromptTemplate(
        input_variables=["vector_result", "query"],
        template="Using this data {vector_result}, answer the following question : {query}"
    )


    chain = prompt | llm

    final_answer = chain.invoke({"vector_result": vector_result, "query": query})

    return final_answer


In [18]:
answer = user_answer("explane me about Worker Reallocation in the AI-Induced Transformation")


print(answer.content)



Okay, let's break down what the provided documents say about worker reallocation in the AI-induced transformation.

**Key Concepts**

*   **AI Exposure:** The degree to which a job's tasks can be automated or augmented by AI.
*   **AI Complementarity:** The degree to which a job's tasks work well alongside AI, where human skills are still valuable and enhanced by AI.
*   **Worker Reallocation:** The movement of workers between different occupations or sectors in response to changes in the labor market.

**Main Points from the Documents**

1.  **Adaptation is Necessary:** The documents emphasize that the labor market will change due to AI. Workers will need to adapt to shifting skill demands and sector shifts. Some will transition to roles that are complementary to AI, while others may struggle.

2.  **Static vs. Dynamic View:** The initial analysis provides a "static picture" of AI exposure based on the current job landscape. However, the document stresses that this is not the end of t